In [ ]:
import pandas as pd
import numpy as np
import pickle

# EPO - MUP Data 

In [ ]:
# Read epo to mup data
df_epovvc = pd.read_csv(r"V:\midp\match\epo2019\epovvc2019.txt"
                        ,sep="\t"
                        ,encoding="utf-8")

In [ ]:
df_epovvc.shape

In [ ]:
df_epovvc.head()

In [ ]:
# One crefo may have many patents (different appln_id/person_id)
df_epovvc.loc[df_epovvc.crefo==2150355978,:]

In [ ]:
# There are only 1s for bestcrefo. No need to drop rows
df_epovvc.bestcrefo.value_counts(dropna=False)

In [ ]:
# Trainings data comprises 36868 firms with known patent match
len(pd.unique(df_epovvc.crefo))

In [ ]:
df_tech = pd.read_csv(r"V:\midp\match\epo2019\tech.txt"
                     ,sep="\t"
                     ,encoding="utf-8")

In [ ]:
# One application ID may be assigned to several technology classes
df_tech.head()

In [ ]:
df_tech.shape

In [ ]:
# There are 35 distinct technology classes
len(pd.unique(df_tech.technology))

In [ ]:
df_ipc = pd.read_csv(r"V:\midp\match\epo2019\ipc.txt"
                     ,sep="\t"
                     ,encoding="utf-8")

In [ ]:
df_ipc.head()

In [ ]:
df_ipc.shape

In [ ]:
# There are 59915 different ipc classes
len(pd.unique(df_ipc.ipc))

In [ ]:
# There are 123 different ipc classes (3-digit level)
len(pd.unique(df_ipc.ipc.apply(lambda x: x[0:3])))

# Merge Data 

In [ ]:
# Merge both datasets
df = df_epovvc.merge(df_tech, how = "left").astype({"technology": "object"}).merge(df_ipc, how = "left")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Write data
with open(r'Q:\Meine Bibliotheken\Research\Projects\Ausgründungen\02_Data\02_Webdata2IPC\df_epovvc_s0.pkl', 'wb') as f:
    pickle.dump(obj=df, file=f)

# Crefo & URL for Scraping 

In [ ]:
df_url = pd.read_csv(r"I:\!Projekte\BMBF_TOBI_131308\01_Arbeitspakete\01_Webscraper\Webscraper\URLs\2019_lebende\MUP_2019_URLs_raw.txt"
                    ,sep="\t"
                    ,encoding="latin-1"
                    ,header=None
                    ,names=['crefo', 'url'] )

In [ ]:
# Strip whitespaces in url column
df_url["url"] = df_url.url.apply(lambda x: x.strip())

In [ ]:
# Merge with crefos
df_crefo = pd.DataFrame(pd.unique(df.crefo), columns={"crefo"}).merge(df_url, how = "left")

In [ ]:
# Filter only crefos with url
df_crefo = df_crefo.loc[df_crefo.url.notnull(),:]

In [ ]:
# Write data
df_crefo.to_csv(r"Q:\Meine Bibliotheken\Research\Projects\Ausgründungen\02_Data\02_Webdata2IPC\urls.txt"
               ,sep = "\t"
               ,encoding="utf-8"
               ,index=False)